In [2]:
import numpy as np
import torch
import random

import logging
import os
import random

import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

In [34]:
target = torch.tensor([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])
print(target.shape)
target

torch.Size([5, 5])


tensor([[0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [1., 1., 0., 0., 1.]])

In [32]:
target = torch.rand(5,5)
print(target)
target.shape

tensor([[0.4687, 0.1387, 0.6461, 0.5121, 0.2060],
        [0.0882, 0.7572, 0.0049, 0.3728, 0.4143],
        [0.8104, 0.5477, 0.8381, 0.5874, 0.2776],
        [0.0821, 0.7569, 0.2585, 0.8314, 0.4017],
        [0.1114, 0.4795, 0.4871, 0.5231, 0.0732]])


torch.Size([5, 5])

In [95]:
from torch.nn import functional as F

def _apply_transformations(img, target, transforms_list):
    
    # apply identical transformations to image and target
    
    if 'RandomVerticalFlip' in transforms_list and random.random() > 0.5:
        print('vflip')
        img = transforms.functional_pil.vflip(img)
        target = transforms.functional_pil.vflip(target)

    if 'RandomHorizontalFlip' in transforms_list and random.random() > 0.5:
        print('hflip')
        img = transforms.functional_pil.hflip(img)
        target = transforms.functional_pil.hflip(target)
        
    if 'RandomCrop' in transforms_list:
        tw = 200
        th = 200
        w,h = img.size
        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
        
        img = img.crop((x1, y1, x1 + tw, y1 + th))
        target = target.crop((x1, y1, x1 + tw, y1 + th))
        
    return img, target
        
    

class ImageDataset2(Dataset):
    """
    Defines object that represents an image Dataset. Combines target and image into one tensor before performing
    transformations.
    """

    def __init__(self, img_dir, targets_dir, dataset_type, transforms_list=[]):
        """
        Initialize ImageDataset.

        Args:
            img_dir (str): Directory for images
            targets_dir (str): Directory for targets related to given images
            transform (transformation to perform on both images and targets):
        """
        self.img_dir = img_dir
        self.targets_dir = targets_dir
        self.transforms_list = transforms_list
        self.dataset_type = dataset_type

        # prepare image list
        img_list = os.listdir(img_dir)
        img_list.sort()
        img_list.remove('.DS_Store')  # remove mac generated files
        self.img_list = [os.path.join(img_dir, img_name) for img_name in img_list]
        
        # prepare target list
        # target image name matches image but also includes suffix
        self.target_list = [os.path.join(targets_dir, img_name[:-4] + '_anno.bmp') for img_name in img_list]

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):

        # load image and target
        img = Image.open(self.img_list[idx]).convert('RGB')
        target = Image.open(self.target_list[idx])
        
        # apply transformations
        if self.dataset_type == 'train':
            # apply random transformations to image and target for training set only
            img, target = _apply_transformations(img, target, self.transforms_list)
        else:
            # crop the image in a standard way?
            pass

        # convert to tensors
        tensor_img = transforms.ToTensor()(img)
        tensor_target = transforms.ToTensor()(target)  # (C, H, W)
  
        # keep only first channel because all three channels are given the same value
        tensor_target_first_channel = tensor_target.squeeze(0)  # (H,W)

        # convert all nonzero target values to 1
        # nonzero values indicate segment
        # zero values indicate background
        tensor_target_first_channel[tensor_target_first_channel != 0] = 1.0

        # convert target to long datatype to indicate classes
        tensor_target_first_channel = tensor_target_first_channel.to(torch.long)

        return tensor_img, tensor_target_first_channel


In [96]:
ids = ImageDataset2('/Users/ryanqnelson/Desktop/test/data/warwick/b/','/Users/ryanqnelson/Desktop/test/data/warwick/b_anno/','train',['RandomCrop'])

In [97]:
tensor_img, tensor_target_first_channel = ids.__getitem__(0)
print(tensor_img.shape, tensor_target_first_channel.shape)

torch.Size([3, 200, 200]) torch.Size([200, 200])


In [6]:
img_dir = '/Users/ryanqnelson/Desktop/test/data/warwick/training/'
img_list = os.listdir(img_dir)
img_list.sort()
img_list.remove('.DS_Store')  # remove mac generated files
img_listT = [os.path.join(img_dir, img_name) for img_name in img_list]
# print(img_listT)

img_dir = '/Users/ryanqnelson/Desktop/test/data/warwick/a/'
img_list = os.listdir(img_dir)
img_list.sort()
img_list.remove('.DS_Store')  # remove mac generated files
img_listA = [os.path.join(img_dir, img_name) for img_name in img_list]
# print(img_listA)

img_dir = '/Users/ryanqnelson/Desktop/test/data/warwick/b/'
img_list = os.listdir(img_dir)
img_list.sort()
img_list.remove('.DS_Store')  # remove mac generated files
img_listB = [os.path.join(img_dir, img_name) for img_name in img_list]
# print(img_listB)

images = img_listT + img_listA + img_listB
len(images)

165

In [87]:
import numpy as np
import torch
import random

import logging
import os
import random

import torch
from PIL import Image
import torchvision.transforms as transforms


def calc_channel_sums(img_path):
    img = Image.open(img_path).convert('RGB')

    # convert to tensor
    tensor_img = transforms.ToTensor()(img)

    # sum pixel values for each channel separately
    means_per_channel = torch.mean(tensor_img, dim=[1, 2])
    stds_per_channel = torch.std(tensor_img, dim=[1, 2])

    return means_per_channel, stds_per_channel


def calc_images_means_stds(img_path_list):
    means = []
    stds = []
    for img_path in img_path_list:
        means_per_channel, stds_per_channel = calc_channel_sums(img_path)
        means.append(means_per_channel)
        stds.append(stds_per_channel)

    # create single tensor of each result
    m = torch.vstack(means)
    s = torch.vstack(stds)
    
    # divide by number of results
    channel_means = torch.sum(m,dim=0) / len(means)
    channel_stds = torch.sum(s,dim=0) / len(stds)
    return channel_means, channel_stds



In [88]:
calc_images_means_stds(images)

(tensor([0.7853, 0.5147, 0.7834]), tensor([0.1566, 0.2124, 0.1169]))